In [1]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
import warnings 
warnings.filterwarnings("ignore")

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Dataset and Create Dataloader

In [2]:
# Load training dataset, only the first 50,000 examples
train_dataset = load_dataset("wmt16", "de-en", split="train[:50000]")

# Load validation and test datasets
validation_dataset = load_dataset("wmt16", "de-en", split="validation")
test_dataset = load_dataset("wmt16", "de-en", split="test")

In [3]:
len(train_dataset), len(validation_dataset), len(test_dataset)

(50000, 2169, 2999)

# Setup- 2B

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

prefix = "translate English to German: "
input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Das Haus ist wunderbar.


### Compute the scores


In [5]:
import sacrebleu

hypotheses = ["hello there general kenobi","foo bar foo bar"]
references = [
    "hello there is general hello",
    "foo bar foo bar"
]
ret = sacrebleu.corpus_bleu(hypotheses, [references])
print(ret)
# Extract the BLEU-1, BLEU-2, BLEU-3, and BLEU-4 scores
print(ret.precisions)

BLEU = 54.54 87.5/66.7/50.0/50.0 (BP = 0.882 ratio = 0.889 hyp_len = 8 ref_len = 9)
[87.5, 66.66666666666667, 50.0, 50.0]


In [6]:
import sacrebleu
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import nltk
nltk.download('wordnet')
nltk.download('punkt')

def calculate_bleu(predicted_corpus, reference_corpus):
    return sacrebleu.corpus_bleu(predicted_corpus, [[i] for i in reference_corpus])

def calculate_meteor(predicted_corpus, reference_corpus):
    m_score = 0
    for line in zip(predicted_corpus, reference_corpus):
        hypo = word_tokenize(line[0])
        ref = word_tokenize(line[1])
        # m_score += meteor_score.meteor_score([ref], hypo)
        m_score += meteor_score.single_meteor_score(ref, hypo)
    return m_score / len(reference_corpus)

def calculate_bert_score(predicted_corpus, reference_corpus):
    p, r, f1 = score(predicted_corpus, reference_corpus, lang="de")
    return p.mean().item(), r.mean().item(), f1.mean().item()

[nltk_data] Downloading package wordnet to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/disk1/sumdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Translate Engligh to German Language

In [7]:
## Translate English to German
def translate_engligh_to_german(dataset):
    reference_corpus = []
    predicted_corpus = []

    for example in dataset:
        input_text = prefix + example["translation"]["en"]
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids
        outputs = model.generate(input_ids)
        predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        reference_corpus.append(example["translation"]["de"])
        predicted_corpus.append(predicted_text)
    return reference_corpus, predicted_corpus

### Evaluation score of T5-model on the Validation set

In [8]:
reference_corpus_val, predicted_corpus_val = translate_engligh_to_german(validation_dataset)

bleu_score_ = calculate_bleu(predicted_corpus_val, reference_corpus_val)
meteor_score_ = calculate_meteor(predicted_corpus_val, reference_corpus_val)
bert_score_ = calculate_bert_score(predicted_corpus_val, reference_corpus_val)

print(f'BLEU Score: {bleu_score_.score}')
print(f'BLEU-1: {bleu_score_.precisions[0]}')
print(f'BLEU-2: {bleu_score_.precisions[1]}')
print(f'BLEU-3: {bleu_score_.precisions[2]}')
print(f'BLEU-4: {bleu_score_.precisions[3]}')
print(f'METEOR Score: {meteor_score_}')
print(f'BERT Scores:')
print(f'Precision in BERT Score: {bert_score_[0]}')
print(f'Recall in BERT Score: {bert_score_[1]}')
print(f'F1 Score in BERT Score: {bert_score_[2]}')

BLEU Score: 39.281465090051306
BLEU-1: 100.0
BLEU-2: 50.0
BLEU-3: 28.571428571428573
BLEU-4: 16.666666666666668
METEOR Score: 0.3697782590377402
BERT Scores:
Precision in BERT Score: 0.8268641829490662
Recall in BERT Score: 0.7662411332130432
F1 Score in BERT Score: 0.7942118644714355


In [9]:
for i in range(5):
    print(f"Example {i+1}")
    print(f"Original: {reference_corpus_val[i]}")
    print(f"Translated: {predicted_corpus_val[i]}")

Example 1
Original: Die Premierminister Indiens und Japans trafen sich in Tokio.
Translated: In Tokio treffen sich die Premierminister Indiens und Japans
Example 2
Original: Indiens neuer Premierminister Narendra Modi trifft bei seinem ersten wichtigen Auslandsbesuch seit seinem Wahlsieg im Mai seinen japanischen Amtskollegen Shinzo Abe in Toko, um wirtschaftliche und sicherheitspolitische Beziehungen zu besprechen.
Translated: Der neue indische Premierminister Narendra Modi trifft sein japanisches Am
Example 3
Original: Herr Modi befindet sich auf einer fünftägigen Reise nach Japan, um die wirtschaftlichen Beziehungen mit der drittgrößten Wirtschaftsnation der Welt zu festigen.
Translated: Herr Modi ist auf einer fünftägigen Reise nach Japan, um die wirtschaftliche
Example 4
Original: Pläne für eine stärkere kerntechnische Zusammenarbeit stehen ganz oben auf der Tagesordnung.
Translated: Auf der Tagesordnung stehen Pläne für eine verstärkte nukleare Zusammenarbeit.
Example 5
Original:

### Evaluation score of T5-model on the Test set

In [10]:
reference_corpus_test, predicted_corpus_test = translate_engligh_to_german(test_dataset)

bleu_score_ = calculate_bleu(predicted_corpus_test, reference_corpus_test)
meteor_score_ = calculate_meteor(predicted_corpus_test, reference_corpus_test)
bert_score_ = calculate_bert_score(predicted_corpus_test, reference_corpus_test)

print(f'BLEU Score: {bleu_score_.score}')
print(f'BLEU-1: {bleu_score_.precisions[0]}')
print(f'BLEU-2: {bleu_score_.precisions[1]}')
print(f'BLEU-3: {bleu_score_.precisions[2]}')
print(f'BLEU-4: {bleu_score_.precisions[3]}')
print(f'METEOR Score: {meteor_score_}')
print(f'BERT Scores:')
print(f'Precision in BERT Score: {bert_score_[0]}')
print(f'Recall in BERT Score: {bert_score_[1]}')
print(f'F1 Score in BERT Score: {bert_score_[2]}')

BLEU Score: 0.0
BLEU-1: 66.66666666666667
BLEU-2: 25.0
BLEU-3: 25.0
BLEU-4: 0.0
METEOR Score: 0.3889479564070389
BERT Scores:
Precision in BERT Score: 0.8345420956611633
Recall in BERT Score: 0.7703850269317627
F1 Score in BERT Score: 0.8000087141990662


In [11]:
for i in range(5):
    print(f"Example {i+1}")
    print(f"Original: {reference_corpus_test[i]}")
    print(f"Translated: {predicted_corpus_test[i]}")

Example 1
Original: Obama empfängt Netanyahu
Translated: Obama erhält Netanjahu
Example 2
Original: Das Verhältnis zwischen Obama und Netanyahu ist nicht gerade freundschaftlich.
Translated: Die Beziehung zwischen Obama und Netanjahu ist nicht gerade freundlich.
Example 3
Original: Die beiden wollten über die Umsetzung der internationalen Vereinbarung sowie über Teherans destabilisierende Maßnahmen im Nahen Osten sprechen.
Translated: Die beiden wollten über die Umsetzung des internationalen Abkommens und über Teherans
Example 4
Original: Bei der Begegnung soll es aber auch um den Konflikt mit den Palästinensern und die diskutierte Zwei-Staaten-Lösung gehen.
Translated: Das Treffen sollte auch den Konflikt mit den Palästinensern und die ums
Example 5
Original: Das Verhältnis zwischen Obama und Netanyahu ist seit Jahren gespannt.
Translated: Die Beziehungen zwischen Obama und Netanjahu sind seit Jahren angespannt.
